# 以TP53为Target 蛋白进行Drug discovery分析

采用ChEMBL数据库，获取实验数据

## 下载数据

导入依赖包

In [ ]:
import pandas as pd
from chembl_webresource_client.new_client import new_client

检索`TP53`在ChEMBL中的记录，选择正确的ID

In [ ]:
# Target search for TP53
target = new_client.target
#Tumour suppressor p53/oncoprotein Mdm2： CHEMBL1907611
target_query = target.search('TP53')
targets = pd.DataFrame.from_dict(target_query)
targets

In [ ]:
activity = new_client.activity
res = activity.filter(target_chembl_id='CHEMBL1907611').filter(standard_type="IC50")

In [ ]:
df = pd.DataFrame.from_dict(res)
df.head(10)

输出数据

In [ ]:
df.to_csv('TP53_activity_data.csv', index=False)

In [ ]:
df.shape

## 数据预处理

去除NA值：

+ IC50

+ 分子式

In [ ]:
#IC50 value
df2 = df[df.standard_value.notna()]
# 分子式
df2 = df2[df.canonical_smiles.notna()]
df2

去除重复

In [ ]:
len(df2.canonical_smiles.unique())

In [ ]:
df2_nr = df2.drop_duplicates(['canonical_smiles'])
df2_nr

选择需要的数据信息

In [ ]:
selection = ['molecule_chembl_id','canonical_smiles','standard_value']
df3 = df2_nr[selection]
df3

In [ ]:
df3.to_csv('TP53_activity_data_preprocessed.csv', index=False)

### Simply EDA

In [ ]:
df3=pd.read_csv("TP53_activity_data_preprocessed.csv")

bioactivity_threshold = []
for i in df3.standard_value:
  if float(i) >= 2270:
    bioactivity_threshold.append("inactive")
  elif float(i) <= 5:
    bioactivity_threshold.append("active")
  else:
    bioactivity_threshold.append("intermediate")

bioactivity_class = pd.Series(bioactivity_threshold, name='class')
df4 = pd.concat([df3, bioactivity_class], axis=1)
df4

In [ ]:
df4.standard_value.describe()

boxplot图

In [ ]:
import seaborn as sns
sns.set_theme(style="white")

In [ ]:
ax = sns.boxplot(y="standard_value", data=df4)
ax = sns.stripplot(y="standard_value",data=df4, color=".25")

密度分布图

In [ ]:
#sns.histplot(df4['standard_value'])
sns.distplot(df4['standard_value'])

In [ ]:
import numpy as np
#sns.histplot(np.log2(df4['standard_value']))
sns.distplot(np.log2(df4['standard_value']))

### 计算 Lipinski 

基于compound的分子式，进行druglikness的估算：

The Lipinski's Rule stated the following:
* Molecular weight < 500 Dalton
* Octanol-water partition coefficient (LogP) < 5
* Hydrogen bond donors < 5
* Hydrogen bond acceptors < 10 

 a set of rule-of-thumb for evaluating the druglikeness of compounds

In [ ]:
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski

In [ ]:
def lipinski(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem) 
        moldata.append(mol)
       
    baseData= np.arange(1,1)
    i=0  
    for mol in moldata:        
       
        desc_MolWt = Descriptors.MolWt(mol)
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_NumHDonors = Lipinski.NumHDonors(mol)
        desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)
           
        row = np.array([desc_MolWt,
                        desc_MolLogP,
                        desc_NumHDonors,
                        desc_NumHAcceptors])   
    
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1      
    
    columnNames=["MW","LogP","NumHDonors","NumHAcceptors"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors

In [ ]:
df_lipinski = lipinski(df4.canonical_smiles)
df_lipinski

In [ ]:
df_combined = pd.concat([df4,df_lipinski], axis=1)
df_combined

### IC50值转换

将原有的IC50进行LOG转换

In [ ]:
import numpy as np
df_combined['logIC50']=[np.log10(m+1) for m in df_combined['standard_value']]
df_combined

In [ ]:
df_combined.logIC50.describe()

In [ ]:
df_combined.to_csv('TP53_activity_data_3class_pIC50.csv')

### 可视化

In [ ]:
import matplotlib.pyplot as plt

df_2class=df_combined[df_combined['class']!="intermediate"]

plt.figure(figsize=(5.5, 5.5))

sns.scatterplot(x='MW', y='LogP', data=df_2class, hue='class', size='logIC50', edgecolor='black', alpha=0.7)

plt.xlabel('MW', fontsize=14, fontweight='bold')
plt.ylabel('LogP', fontsize=14, fontweight='bold')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)
plt.savefig('plot_MW_vs_LogP.pdf')

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'class', y = 'MW', data = df_2class)

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('MW', fontsize=14, fontweight='bold')

plt.savefig('plot_MW.pdf')

## 特征提取

采用padel进行化合物fingerprint计算


In [ ]:
import pandas as pd
df=pd.read_csv("TP53_activity_data_3class_pIC50.csv",index_col=0)
df

In [ ]:
selection = ['canonical_smiles','molecule_chembl_id']
df_selection = df[selection]
df_selection.to_csv('molecule.smi', sep='\t', index=False, header=False)

In [ ]:
df_selection

每个化合物，共获取800多个features


### 使用python计算fingerPrint

In [ ]:
from padelpy import from_smiles

In [ ]:
fingerprints = from_smiles('CCCN1C(=O)c2ccccc2C1(NC(=O)c1ccc(C(C)(C)C)cc1)c1ccc(OCOCC[Si](C)(C)C)cc1', fingerprints=True, descriptors=False)

**批量预测**

In [ ]:
from padelpy import padeldescriptor

In [ ]:
# to calculate PubChem fingerprints
padeldescriptor(mol_dir="molecule.smi", 
                d_file="descriptors.csv",
                fingerprints=True,
                removesalt=True,
                standardizenitro=True,
                threads=2,
                log=True
               )

### 将features与因变量结合

In [ ]:
padel_features= pd.read_csv('descriptors.csv')
padel_features

In [ ]:
padel_featuresX=padel_features.drop(columns=['Name'])
dataset = pd.concat([padel_featuresX,df['logIC50']], axis=1)
dataset

In [ ]:
dataset.to_csv('TP53_Features_fp_IC50.csv', index=False)

## 建立ML model

In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [ ]:
df = pd.read_csv('TP53_Features_fp_IC50.csv')
df

### X,Y features提取

In [ ]:
X = df.drop('logIC50', axis=1)
X

In [ ]:
Y = df.logIC50
Y

### Features过滤

可选

In [ ]:
#from sklearn.feature_selection import VarianceThreshold
#selection = VarianceThreshold()    
#X = selection.fit_transform(X)
#X.shape

### Dataset 划分

从中获取train & test 数据集

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
X_train.shape,X_test.shape

### Random Forest Regression Model

In [ ]:
import numpy as np
import random
np.sqrt(515)

In [ ]:
model = RandomForestRegressor(n_estimators=25)
model.fit(X_train, Y_train)
r2 = model.score(X_test, Y_test)
r2

### 可视化

In [ ]:
Y_pred = model.predict(X_test)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(color_codes=True)
sns.set_style("white")

ax = sns.regplot(Y_test, Y_pred, scatter_kws={'alpha':0.2})
ax.set_xlabel('Experimental IC50')
ax.set_ylabel('Predicted IC50')

plt.show

In [ ]:
r=np.corrcoef(Y_test,Y_pred)
r[0, 1]

In [ ]:
import scipy.stats
scipy.stats.pearsonr(Y_test, Y_pred)

## 比较不同Models

主要是使用 [lazypredict](https://github.com/shankarpandala/lazypredict) 实现多个ML Models的比较

In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
import lazypredict
from lazypredict.Supervised import LazyRegressor

### 分析结果

In [ ]:
mlf = LazyRegressor(verbose=0, ignore_warnings=True, custom_metric=None)
models_x,predictions = mlf.fit(X_train, X_train, Y_train, Y_train)

In [ ]:
predictions.head(10)

### 结果可视化

做四幅图进行结果可视化

In [ ]:
df=predictions.head(10)
df

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

#sns.set(color_codes=True)
#sns.set_style("white")
fig=plt.figure(figsize=(15,8))
grid=plt.GridSpec(2,2,wspace=1,hspace=0.5,figure=fig)

ax1=fig.add_subplot(grid[0,0])
ax2=fig.add_subplot(grid[0,1])
ax3=fig.add_subplot(grid[1,0])
ax4=fig.add_subplot(grid[1,1])

sns.barplot(data=df, x='Adjusted R-Squared', y=df.index,ax=ax1)
sns.barplot(data=df, x='R-Squared', y=df.index,ax=ax2)
sns.barplot(data=df, x='RMSE', y=df.index,ax=ax3)
sns.barplot(data=df, x='Time Taken', y=df.index,ax=ax4)

plt.show()

Top3 best models:

+ DecisionTreeRegressor

+ ExtraTreeRegressor

+ ExtraTreesRegressor

Top3 best in test data:

+ RandomForestRegressor

+ GradientBoostingRegressor

+ LGBMRegressor

## Model的应用

对于一个未知化合物的预测：



In [ ]:
from padelpy import from_smiles

In [ ]:
x="CCCN1C(=O)c2ccccc2C1(NC(=O)c1ccccc1)c1ccc(C(C)(C)C)cc1"
fingerprints = from_smiles(x, fingerprints=True, descriptors=False)
fingerprints

In [ ]:
#array转化
vs=[]
for k,v in fingerprints.items():
    vs.append(int(v))
vs=np.array(vs).reshape(1,-1)

In [ ]:
Y_pred = model.predict(vs)
Y_pred

## 环境迁移

在windows上进行环境迁移，我又又又采坑了！！

### 导出环境

```sh
conda env export > drug.yaml
```

### 导出依赖包

```sh
pip freeze > requirements.txt
```

### 坑s

#### 坑1

以上均是常规操作，但是要注意`drug.yaml` 和 `requirements.txt` 文件中一些很不合适的 packages比如这些：

+ requirements.txt

```
cycler @ file:///home/conda/feedstock_root/build_artifacts/cycler_1635519461629/work

fonttools @ file:///D:/bld/fonttools_1639926487978/work
```
对于这些怎么办，我通过手动`pip show cycler`去找到对应的版本信息

#### 坑2

`drug.yaml` 文件中有pip包信息，会和`requirements.txt`中的包发送冲突。

添加channel，并删除`-pip:`下的所有包

#### 坑3

使用系统自带的`cmd`无法正常安装conda 环境，但是使用conda自带的`prompt`就可以！！！

```sh
pip install -r .\requirements.txt --ignore-installed -i https://pypi.douban.com/simple/
```

#### 无限坑

各种包版本的依赖和冲突，需要手动解决

### 总结

这两个文件运行比较稳定：

+ ./drug.yaml

+ ./requirements.txt

而在envs/下的两个文件，则是最为原始的环境文件，不能成功运行